In [5]:
!pip install "mlflow<2.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 25.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 30.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.8/171.8 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 28.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 30.2 MB/s eta 0:00:00a 

In [1]:
from minio import Minio

minio_client = Minio(
    "localhost:57100",
    access_key="mlops",
    secret_key="mlops123",
    secure=False,
)

if not minio_client.bucket_exists('mlflow'):
    minio_client.make_bucket('mlflow')

In [2]:
import mlflow

mlflow.set_tracking_uri("http://localhost:59000")

for i in range(3):
    with mlflow.start_run():
        mlflow.log_param("trial", i)
        mlflow.log_metric("metric", i+1)

In [3]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression

import numpy as mp
import pandas as pd

class MyModel:

    def __init__(self, n_features):
        self.n_features = n_features
        self.model = AdaBoostRegressor(random_state=0, n_estimators=100)

    def fit(self):
        x, y = make_regression(
            n_features=self.n_features,
            n_informative=2,
            random_state=0,
            shuffle=False
        )
        self.model.fit(x, y)

    def predict(self, input_sample):
        pd_input = pd.DataFrame(input_sample)
        pred = self.model.predict(pd_input)
        return pd.DataFrame(pred)

input_df = pd.DataFrame(data=[[0, 0.0]], columns=["co2", "temperature"])

# create and train model
my_model = MyModel(
    n_features=len(input_df.columns)
)

In [4]:
mlflow.sklearn.autolog()

with mlflow.start_run() as run:
    my_model.fit()

2025/01/07 21:40:56 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: API request to http://localhost:59000/api/2.0/mlflow-artifacts/artifacts/0/4f6cb4e4b7244905ad40f5d151013872/artifacts/estimator.html failed with exception HTTPConnectionPool(host='localhost', port=59000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/0/4f6cb4e4b7244905ad40f5d151013872/artifacts/estimator.html (Caused by ResponseError('too many 500 error responses'))
